# Kakao API를 이용한 지오코딩(2023/12/24)
<br> 기존의 프로젝트는 python 내의 geopy 라이브러리를 이용하여 지오코딩을 실시하였음. 그러나 대용량의 data를 처리함에 있어서 geopy는 처리 시간이 적지 않게 소요되었음. 이에 대한 또 다른 해결책으로 Kakao api를 이용하여 지오코딩을 할 수 있는 방법을 제시하고자 함.<br>아래의 코드는 위도와 경도가 누락된 csv 파일을 kakao api를 통해 위도와 경도 칼럼을 추가한 새로운 csv로 저장하는 코드임.

우선 Kakao api 키를 발급받고, 아래의 코드에서 headers의 안에 자신의 고유키를 삽입함.<br>( https://developers.kakao.com/ )

In [1]:
import requests, json

def Kakao_Geocoding(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    # 주소를 string으로 받아서 위경도 좌표를 list로 반환함.
    # 'KaKaoAK ' 뒤에 개인키만 입력함.
    # ex) KakaoAK 21ba322daf2d740f327e6b8316b81333
    headers = {"Authorization": "KakaoAK 21ba322daf2d740f327e6b8316b81857"}
    # try , except 문법을 추가하여 에러가 나더라도 프로그램이 진행될 수 있도록 설계함.
    # 에러가 발생하면 [0,0]을 위경도로 설정함.
    try:
        api_json = json.loads(str(requests.get(url,headers=headers).text))
        address = api_json['documents'][0]['address']
        coordinate = [str(address['x']), str(address['y'])]
        address_name = address['address_name']
        return coordinate
    
    except:
        return [0,0]
    
# kakaomap api를 이용하여 에러가 발생한 주소의 위경도 값을 추출함.
Kakao_Geocoding('서울특별시 동대문구 보문로 6')

['127.023530607859', '37.5761363837871']

'서울특별시 동대문구 보문로 6'를 입력 parameter로 주었을 때, 위경도 좌표값이 정상적으로 나오는 것을 확인함.

이제부터는 위경도 좌표값이 누락된 csv 파일을 열어서 지오코딩을 실시한 후, '위도', '경도' 칼럽을 추가해 새로운 csv로 저장하는 과정임.

pandas를 import한 후에<br>
지오코딩을 실시하고자 하는 csv 파일명을 pd.read_csv의 input parameter로 설정함.

In [2]:
import pandas as pd
# 변수명 = pd.read_csv('파일명.csv', encoding='cp949') / # test_raw는 임시로 설정한 변수임. 
# encoding은 보통 'utf8' or 'cp949'로 설정하면 정상적으로 인식할 수 있음. 
test_raw = pd.read_csv('.\\data\\testdata_beforeCoding.csv', encoding='cp949')
test_raw

,행정구역명,학원/교습소,학원명,도로명주소,도로명상세주소,분야명,정원합계
0,동대문구,학원,한국이용학원,서울특별시 동대문구 왕산로33길 32,", 2층 (제기동)",직업기술,52
1,동대문구,학원,대성회계실무학원,서울특별시 동대문구 왕산로 225,", 5층2호?5층6호?5층7호 중 일부(B호) (청량리동, 미주상가)",종합(대),404
2,동대문구,학원,국제전자과학학원,서울특별시 동대문구 고산자로36길 3,", 3층 일부 (제기동, 경동시장)",직업기술,30
3,동대문구,학원,조이앤플레이음악학원,서울특별시 동대문구 이문로 38,", 3층 301호 (휘경동, 민족통일빌딩)",예능(대),98
4,동대문구,학원,한독자동차정비학원,서울특별시 동대문구 왕산로 31,", 2층,3층 (신설동)",직업기술,210
...,...,...,...,...,...,...,...
124,동대문구,교습소,프라하피아노교습소,서울특별시 동대문구 고산자로30길 70,1층일부 (용두동),예능(대),20
125,동대문구,교습소,e해법수학교습소,서울특별시 동대문구 서울시립대로 31,동아아파트 102동 207호 (전농동),입시.검정 및 보습,21
126,동대문구,교습소,쥴리어드음악교습소,서울특별시 동대문구 답십리로23나길 17,", 1층일부 (전농동)",예능(대),25
127,동대문구,교습소,청솔해법영어교습소,서울특별시 동대문구 전농로10길 20,", 201호, 202호 (답십리동,답십리청솔우성아파트상가)",입시.검정 및 보습,40


현재 test_raw의 데이터에는 도로명주소만 있을 뿐, 위도와 경도 값이 없는 것을 확인할 수 있음.

다음으로 test_raw에서 칼럼 '도로명주소'의 값들을 추출함. 지오코딩의 input parameter로 넣기 위한 데이터 추출과정임.

In [12]:
# 대용량 데이터를 지오코딩 할 때 약간의 시간이 소요됨.
# 지오코딩이 어느정도 진행되었는지 시각적으로 표현할 수 있는 library임.
!pip install tqdm

In [11]:
# codingList = test_raw['도로명주소'] csv 파일에서 도로명주소 또는 구 주소를 담고 있는 칼럼명을 적어줌.
# test_raw에서는 '도로명주소'라는 칼럼이 주소를 가지고 있음.
CodingList = test_raw['도로명주소']

# 도로명주소를 기반으로 지오코딩 실시
# lat은 각 record의 위도를 element로 가지는 list
# long은 각 record의 경도를 element로 가지는 list

lat = []
long = []
from tqdm import tqdm
for k in tqdm(CodingList):
    long.append(Kakao_Geocoding(k)[0])
    lat.append(Kakao_Geocoding(k)[1])

100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [00:20<00:00,  6.29it/s]


이렇게 추출한 리스트 lat, long 과 기존의 데이터에서 사용하고자 하는 칼럼으로 새로운 dataframe를 생성함.

In [13]:
Final_data = pd.DataFrame({'행정구역': test_raw['행정구역명'],
                                      '학원이름' : test_raw['학원명'],
                                      '주소':CodingList,
                                      '상세주소':test_raw['도로명상세주소'],
                                      '위도':lat,'경도':long,'분야명':test_raw['분야명']})
Final_data

,행정구역,학원이름,주소,상세주소,위도,경도,분야명
0,동대문구,한국이용학원,서울특별시 동대문구 왕산로33길 32,", 2층 (제기동)",37.5807438360293,127.041917061106,직업기술
1,동대문구,대성회계실무학원,서울특별시 동대문구 왕산로 225,", 5층2호?5층6호?5층7호 중 일부(B호) (청량리동, 미주상가)",37.5821227010138,127.047209103701,종합(대)
2,동대문구,국제전자과학학원,서울특별시 동대문구 고산자로36길 3,", 3층 일부 (제기동, 경동시장)",37.5790377655561,127.039142935818,직업기술
3,동대문구,조이앤플레이음악학원,서울특별시 동대문구 이문로 38,", 3층 301호 (휘경동, 민족통일빌딩)",37.590757612557,127.056680204054,예능(대)
4,동대문구,한독자동차정비학원,서울특별시 동대문구 왕산로 31,", 2층,3층 (신설동)",37.5769771989632,127.026873164406,직업기술
...,...,...,...,...,...,...,...
124,동대문구,프라하피아노교습소,서울특별시 동대문구 고산자로30길 70,1층일부 (용두동),37.575230527702,127.042015853524,예능(대)
125,동대문구,e해법수학교습소,서울특별시 동대문구 서울시립대로 31,동아아파트 102동 207호 (전농동),37.5753105451014,127.045864474612,입시.검정 및 보습
126,동대문구,쥴리어드음악교습소,서울특별시 동대문구 답십리로23나길 17,", 1층일부 (전농동)",37.5800228229668,127.052792227116,예능(대)
127,동대문구,청솔해법영어교습소,서울특별시 동대문구 전농로10길 20,", 201호, 202호 (답십리동,답십리청솔우성아파트상가)",37.5746094744702,127.060129124176,입시.검정 및 보습


'위도', '경도' 칼럼이 추가된 것을 확인할 수 있음.

Kakao_Geocoding에서 try, except을 사용하였기 때문에, 오류가 발생하여 위도와 경도가 0인 객체가 있는지 살펴봄.

In [14]:
Final_data[Final_data['위도']==0]

,행정구역,학원이름,주소,상세주소,위도,경도,분야명


In [15]:
Final_data[Final_data['경도']==0]

,행정구역,학원이름,주소,상세주소,위도,경도,분야명


위도와 경도가 0인 객체가 없다면 지오코딩이 정상적으로 실행된 것이므로 csv 파일로 저장함.

In [16]:
Final_data.to_csv('.\\data\\testdata_afterCoding.csv')